In [2]:
import openai
import os
import json
import sys
from collections import defaultdict
from dotenv import load_dotenv

load_dotenv()

# OpenAI API 키 설정
openai.api_key = os.getenv('OPENAI_API_KEY')

# JSON 파일 로드
with open('./DataSet/User_DataSet.json', 'r') as f:
    data = json.load(f)

# 사용자 이름으로 데이터를 검색하는 함수
def find_user_data(name, data):
    for user in data:
        if user['name'] == name:
            return user
    return None

# 소비 패턴 분석 함수
def analyze_spending(transactions):
    spending = defaultdict(int)
    for transaction in transactions:
        category = transaction[3]
        amount = int(transaction[2])
        spending[category] += amount
    return spending

# 카드 추천 함수
def recommend_card_gpt(spending, user_name):
    spending_summary = "\n".join([f"{category}: {amount}원" for category, amount in spending.items()])

    prompt = (
        f"{user_name}의 소비 패턴은 다음과 같습니다:\n"
        f"{spending_summary}\n\n"
        "이 소비 패턴을 기반으로 추천할 만한 신용카드는 무엇인가요?"
    )

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who recommends credit cards based on spending patterns."},
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content']

# 카테고리, 소비패턴에 따른 카드 추천
def recommend_card_based_on_input(user_input, user_name, user_data):
    # 소비 패턴 분석
    transactions = user_data.get('transactions_Withdrawal', [])
    spending = analyze_spending(transactions)
    spending_summary = "\n".join([f"{category}: {amount}원" for category, amount in spending.items()])

    prompt = (
        f"{user_name}님이 다음과 같은 질문을 하셨습니다:\n"
        f"{user_input}\n\n"
        "이 질문이 신용카드 추천과 관련이 있다면, "
        "이 소비 패턴을 기반으로 추천할 만한 KB국민카드를 알려주세요.\n"
        f"소비 패턴은 다음과 같습니다:\n"
        f"{spending_summary}"
    )

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant who provides recommendations based on user input and spending patterns."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content']

# 일반 질문 처리
def gpt_general_response(user_input):
    # 열받는 기본 출력 문장 삭제
    prompt = f"{user_input}\nAnswer concisely and directly without any preamble."
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message['content']

# 입력에 따른 정보 처리
def handle_user_input(user_name):
    user_data = find_user_data(user_name, data)

    if not user_data:
        return f"{user_name}님의 데이터를 찾을 수 없습니다."

    while True:
        user_input = input(f"{user_name}님, 질문을 입력하세요 ('종료' 입력 시 종료): ")

        if user_input.lower() == '종료':
            print("대화를 종료합니다.")
            break

        # 카테고리 및 카드 추천 처리
        recommendation = recommend_card_based_on_input(user_input, user_name, user_data)

        if "추천" in recommendation or "카드" in recommendation:
            print(recommendation)
        else:
            # 일반 질문 처리
            response = gpt_general_response(user_input)
            print(response)

        sys.stdout.flush()

# 사용 시작
# user_name = input("사용자의 이름을 입력하세요: ")
# handle_user_input(user_name)

# 버퍼를 비웠음에도 입력 후 공백 답변 혹은 다음 답변을 주어야 제대로 된 기존 질문 답변 출력됨

최윤호님의 소비 패턴을 고려할 때, KB국민카드 중에서는 KB Star카드를 추천드립니다. 

KB Star카드는 다양한 소비 혜택과 포인트 적립이 가능한 카드로, 특히 교통, 유흥, 음식점 등 다양한 분야에서 혜택을 받을 수 있습니다. 최윤호님의 소비 패턴에 맞추어 교통 및 영화, 음식점과 같은 소비에서 혜택을 받을 수 있을 것으로 판단됩니다. 또한, 교육에 대한 지출도 많으시기 때문에 교육 관련 혜택도 활용할 수 있습니다.

KB Star카드를 통해 최윤호님의 다양한 소비에 대한 혜택 및 적립 포인트를 누리시길 바랍니다.
대화를 종료합니다.
